In [1]:
import socket
import time
from datetime import datetime
import threading
import pyvisa
import numpy as np

In [2]:
'''
Setup communication
'''
# pip install pyvisa, pyvisa-py'
#find ID's with "lsusb"
#then run this as super user
#echo 'SUBSYSTEM=="usb", ACTION=="add", ATTRS{idVendor}=="0957", ATTRS{idProduct}=="1798" GROUP="tty", MODE="0660"' >> /etc/udev/rules.d/99-com.rules
# sudo usermod -a -G tty pi

#Bus 001 Device 006: ID 05e6:6500 Keithley Instruments Digital Multimeter

#echo 'SUBSYSTEM=="usb", ACTION=="add", ATTRS{idVendor}=="05e6", ATTRS{idProduct}=="6500" GROUP="tty", MODE="0660"' >> /etc/udev/rules.d/99-com.rules
rm = pyvisa.ResourceManager()
rm.list_resources('?*')

inst = rm.open_resource('USB0::1510::25856::04440503::0::INSTR')

inst.write('*IDN?')
print(inst.read())

KEITHLEY INSTRUMENTS,MODEL DMM6500,04440503,1.0.04b



In [4]:
'''
set up backend
'''
import importlib
import serverBackend as sb
importlib.reload(sb)

#add the commands we want to use - see manual
multimter_keys = [
    'SENS:FUNC', # "CURR:DC" "VOLT:DC" etc
    'SENS:CURR:DC:RANG', # current range in A, 0.001, 0.1, 1 etc
    'DISP:CURR:DC:DIG', # digits shown on screen
    'SENS:CURR:DC:NPLC', #This command sets the integration time in number of power line cycles (PLCs) on the specified channels.Integration time affects the measurement resolution (for better resolution, use a longer integration time) and measurement speed (for faster measurements, use a shorter integration time).If you omit the optional <ch_list> parameter, this command applies to the currently defined scan list.
    'READ?',
    ]
sb.setup(inst, multimter_keys)
sb.query('*IDN?')

'KEITHLEY INSTRUMENTS,MODEL DMM6500,04440503,1.0.04b'

In [5]:
print(sb.keys)

['SENS:FUNC', 'SENS:CURR:DC:RANG', 'DISP:CURR:DC:DIG', 'SENS:CURR:DC:NPLC', 'READ?']


In [6]:
sb.set_single_param('SENS:FUNC','"CURR:DC"')
sb.set_single_param('SENS:CURR:DC:RANG','0.001')
sb.set_single_param('SENS:CURR:DC:NPLC','1000')

   set  SENS:FUNC: "CURR:DC"
   set  SENS:CURR:DC:RANG: 0.001
   set  SENS:CURR:DC:NPLC: 1000


In [7]:
sb.query('READ?') # A

'-6.406820E-09'

In [ ]:
serversocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
port = 5008
host = sb.get_ip() #"127.0.0.1"
serversocket.bind((host, port))
serversocket.listen(5)
print ('server started and listening')
while 1:
    clientsocket, address = serversocket.accept()
    sb.server(clientsocket, address)

server started and listening
Current Time = 15:48:23
